In [1]:
#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

import os
#
os.environ["CUDA_VISIBLE_DEVICES"]= gpu6 + "," + gpu5

In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:XLA_GPU:1', '/device:GPU:0', '/device:GPU:1']


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]

'GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f,GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1'

In [ ]:
#Run from google colab

!pip install SimpleITK

from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
%cd /content/gdrive/MyDrive/2 TUM/Project/Internships/notebooks

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import random

In [5]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [6]:
tf.__version__

'2.3.1'

In [ ]:
def myshow(img):
    nda = sitk.GetArrayViewFromImage(img)
    plt.imshow(nda)

In [ ]:
img_t1 = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_t1.nii.gz"))
img_t1ce = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_t1ce.nii.gz"))
img_t2 = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_t2.nii.gz"))
img_flair = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_flair.nii.gz"))
img_seg = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_seg.nii.gz"))

"""
img_t1 = sitk.Cast(img_t1,sitk.sitkFloat32) #to ceonvert png to sitk
img_t1ce = sitk.Cast(img_t1ce,sitk.sitkFloat32) #to ceonvert png to sitk
img_t2 = sitk.Cast(img,sitk_t2.sitkFloat32) #to ceonvert png to sitk
img_flair = sitk.Cast(img_flair,sitk.sitkFloat32) #to ceonvert png to sitk
img_seg = sitk.Cast(img_seg,sitk.sitkFloat32) #to ceonvert png to sitk

print(img.GetSize())
print(img.GetOrigin()) # the top left corner usually
print(img.GetSpacing()) #physical size of each pixel
print(img.GetDirection()) #3D matrix
print(img.GetNumberOfComponentsPerPixel()) #number of channels in one pixel
"""

width = img_t1.GetWidth()
height = img_t1.GetHeight()
depth = img_t1.GetDepth()

nda_t1 = sitk.GetArrayViewFromImage(img_t1)
nda_t1ce = sitk.GetArrayViewFromImage(img_t1ce)
nda_t2 = sitk.GetArrayViewFromImage(img_t2)
nda_flair = sitk.GetArrayViewFromImage(img_flair)
nda_seg = sitk.GetArrayViewFromImage(img_seg)

In [ ]:
npa_zslice_t1 = sitk.GetArrayViewFromImage(img_t1)[int(depth/2),:,:]
npa_zslice_t1ce = sitk.GetArrayViewFromImage(img_t1ce)[int(depth/2),:,:]
npa_zslice_t2 = sitk.GetArrayViewFromImage(img_t2)[int(depth/2),:,:]
npa_zslice_flair = sitk.GetArrayViewFromImage(img_flair)[int(depth/2),:,:]
npa_zslice_seg = sitk.GetArrayViewFromImage(img_seg)[int(depth/2),:,:]

fig = plt.figure(figsize=(10,3))

fig.add_subplot(1,5,1)
plt.imshow(npa_zslice_t1, cmap=plt.cm.Greys_r)
plt.title('T1', fontsize=10)
plt.axis('off')

fig.add_subplot(1,5,2)
plt.imshow(npa_zslice_t1ce, cmap=plt.cm.Greys_r)
plt.title('T1_ce', fontsize=10)
plt.axis('off')

fig.add_subplot(1,5,3)
plt.imshow(npa_zslice_t2, cmap=plt.cm.Greys_r)
plt.title('T2', fontsize=10)
plt.axis('off')

fig.add_subplot(1,5,4)
plt.imshow(npa_zslice_flair, cmap=plt.cm.Greys_r)
plt.title('flair', fontsize=10)
plt.axis('off')

fig.add_subplot(1,5,5)
plt.imshow(npa_zslice_seg, cmap=plt.cm.Greys_r)
plt.title('Segmentation', fontsize=10)
plt.axis('off')

In [7]:
ImgDir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
features_path = list()
labels_path = list()
count = 0
limit = 100

for folder in os.listdir(ImgDir):
    count +=1
    if 'Training' in folder and count<limit:
        new_dir = os.path.join(ImgDir,folder)
        data = os.listdir(new_dir)
        for files in data:
            if 'flair' in files:
                features_path.append(os.path.join(new_dir, files))
            if 'seg' in files:
                labels_path.append(os.path.join(new_dir, files))

print(len(features_path))
print(len(labels_path))

X=features_path
Y=labels_path

99
99


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=1)

In [ ]:
def keras_generator_train_val_test(batch_size, choice="train"):

    if choice == "train":
        X = X_train
        Y = y_train
    elif choice == "val":
        X = X_val
        y = y_val
    else:
        print("Invalid Option")
        return False
    
    while True:
        x_batch = []
        y_batch = []

        num=batch_size
        
        for i in range(batch_size):
            
            rand = random.choice(range(len(X)))

            x_path = X[rand]
            y_path = Y[rand]

            x = sitk.GetArrayViewFromImage(sitk.ReadImage(x_path)) / 255.
            y = sitk.GetArrayViewFromImage(sitk.ReadImage(y_path)) / 255.
            
            #x = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(x_path)),sitk.sitkUInt16))
            #y = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(y_path)),sitk.sitkUInt16))
            
            #x = np.moveaxis(x, [0, 1, 2], [-1, -2, -3])
            #y = np.moveaxis(y, [0, 1, 2], [-1, -2, -3])
            
            
            #x = np.resize(x,(240,240,128,1))
            #y = np.resize(y,(240,240,128,1))

            x_batch.append(x)
            y_batch.append(y)

        x_batch = np.concatenate(np.array(x_batch),0)
        y_batch = np.concatenate(np.array(y_batch),0)
        
        yield x_batch, y_batch

for x, y in keras_generator_train_val_test(100, choice="train"):
    print(x.shape, y.shape)
    break

In [11]:
x_batch = []
y_batch = []

training_img = 100

for i in range(training_img):

    rand = random.choice(range(len(X)))

    x_path = X[rand]
    y_path = Y[rand]

    x = sitk.GetArrayViewFromImage(sitk.ReadImage(x_path)) / 255.
    y = sitk.GetArrayViewFromImage(sitk.ReadImage(y_path)) / 255.

    x_batch.append(x)
    y_batch.append(y)

x_data = np.concatenate(np.array(x_batch),0)
y_data = np.concatenate(np.array(y_batch),0)

In [12]:
import math
import datetime
import tensorflow.python.keras

batch_size = 32

class BRATSSequence(tensorflow.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = X, Y
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

    #def on_epoch_end(self, epoch, logs=None):
        
        #print("epoch finished")

sequence = BRATSSequence(x_data, y_data, batch_size = batch_size)

In [18]:
np.shape(x_data)

(15500, 240, 240)

In [ ]:
# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

import math
#import keras
import datetime
import tensorflow.python.keras

batch_size = 1

class BRATSSequence(tensorflow.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = X, Y
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        x_batch = []
        y_batch = []

        for i in range(training_img):
            
            rand = random.choice(range(len(X)))

            x_path = X[rand]
            y_path = Y[rand]
            
            x = sitk.GetArrayViewFromImage(sitk.ReadImage(x_path)) / 255.
            y = sitk.GetArrayViewFromImage(sitk.ReadImage(y_path)) / 255.

            #x = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(x_path)),sitk.sitkFloat32))
            #y = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(y_path)),sitk.sitkFloat32))
            
            #x = np.moveaxis(x, [0, 1, 2], [-1, -2, -3])
            #y = np.moveaxis(y, [0, 1, 2], [-1, -2, -3])
            
            #x = crop_center(x,64,64,64)
            #y = crop_center(y,64,64,64)

            #x = np.resize(x,(64,64,64,1))
            #y = np.resize(y,(64,64,64,1))
            
            #x = np.resize(x,(240,240,128,1))
            #y = np.resize(y,(240,240,128,1))

            x_batch.append(x)
            y_batch.append(y)

        x_batch = np.concatenate(np.array(x_batch),0)
        y_batch = np.concatenate(np.array(y_batch),0)
        
        print(np.shape(y_batch))
        print(np.shape(x_batch))

        return x_batch, y_batch

    #def on_epoch_end(self, epoch, logs=None):
        
        #print("epoch finished")

sequence = BRATSSequence(X, Y, batch_size = batch_size)

In [19]:
X_train, X_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=1)

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(241)
plt.title('Flair')
plt.axis('off')
plt.imshow(x[:, :, 120])
plt.subplot(242)
plt.title('Ground Truth(Full)')
plt.axis('off')
plt.imshow(y[:, :, 120])

In [13]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
smooth = 0.005

In [25]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def get_model():
    in1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1 ))

    conv1 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.5)(conv1)
    conv1 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.5)(conv2)
    conv2 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.5)(conv3)
    conv3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.5)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.5)(conv6)
    conv6 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.5)(conv7)
    conv7 = Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])   

    model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef])


    return model

In [26]:
model2=get_model()
model2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 1) 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 240, 240, 8)  80          input_2[0][0]                    
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 240, 240, 8)  0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 240, 240, 8)  584         dropout_7[0][0]                  
_______________________________________________________________________________________

In [27]:
validation_tup = (X_val,y_val)

In [28]:
model2.fit(X_train, y_train, batch_size = 32, validation_data =validation_tup, epochs=20, verbose=1, shuffle=True)

Epoch 1/20
436/436 [==============================] - 43s 100ms/step - loss: 0.9999 - dice_coef: 9.4637e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 2/20
436/436 [==============================] - 38s 86ms/step - loss: 1.0000 - dice_coef: 3.4921e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 3/20
436/436 [==============================] - 38s 88ms/step - loss: 1.0000 - dice_coef: 3.5476e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 4/20
436/436 [==============================] - 38s 87ms/step - loss: 1.0000 - dice_coef: 3.4793e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 5/20
436/436 [==============================] - 38s 87ms/step - loss: 1.0000 - dice_coef: 3.4973e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 6/20
436/436 [==============================] - 38s 86ms/step - loss: 1.0000 - dice_coef: 3.5233e-05 - val_loss: 1.0000 - val_dice_coef: 3.9172e-05
Epoch 7/20
436/436 [==============================] - 38s 88ms/step - loss:

KeyboardInterrupt: 

In [ ]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, epoch, logs={}):

        res_dir = "intermediate_results_black_background"

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

        print('Training: epoch {} begins at {}'.format(epoch, datetime.datetime.now().time()))

    def on_epoch_end(self, epoch, logs=None):
        res_dir = "intermediate_results_black_background/"
        print('Training: epoch {} ends at {}'.format(epoch, datetime.datetime.now().time()))
        
        for x_test, y_test in keras_generator_train_val_test(batch_size, choice="test"):
            break
        p = np.reshape(x_test[0], (1, 512, 512, 3))
        prediction = self.model.predict(p)

        x_img = f"{res_dir}{epoch}_X_input.jpg"
        y_img = f"{res_dir}{epoch}_Y_truth.jpg"
        predicted_img = f"{res_dir}{epoch}_Y_predicted.jpg"

        cv2.imwrite(x_img, x_test[0] * 255.)
        cv2.imwrite(y_img, y_test['seg'][0] * 255.)
        cv2.imwrite(predicted_img, prediction[0] * 255.)

In [ ]:
model_name = "..\\data\\Unet_black_background_47_epochs.h5"

batch_size = 2;

#modelcheckpoint = ModelCheckpoint(model_name,
                                  monitor='val_loss',
                                  mode='auto',
                                  verbose=1,
#                                  save_best_only=True)

lr_callback = ReduceLROnPlateau(min_lr=0.000001)

#callback_list = [modelcheckpoint, lr_callback, MyCustomCallback()]

history = model.fit_generator(
    keras_generator_train_val_test(batch_size, choice="train"),
    validation_data = keras_generator_train_val_test(batch_size, choice="val"),
    validation_steps = 100,
    steps_per_epoch=100,
    epochs=epochs,
    verbose=1, 
    shuffle=True
    #callbacks = callback_list,
)

In [ ]:

def keras_generator_train_val_test(batch_size, choice="train"):

    if choice == "train":
        X = X_train
        y = y_train
    elif choice == "val":
        X = X_val
        y = y_val
    else:
        print("Invalid Option")
        return False
        
    while True:
        x_batch = []
        y_batch = []

        for i in range(batch_size):
            
            x = sitk.ReadImage(X[i])
            x_np = sitk.GetArrayViewFromImage(x) / 255.
            y = sitk.ReadImage(y[i])
            y_np = sitk.GetArrayViewFromImage(y) / 255.
            
            x_batch.append(x_np)
            y_batch.append(y_np)

        
        x_batch = np.array(x_batch)
        # y_batch = np.array(y_batch)

        y_batch = {'seg': np.array(y_batch),
                #    'cls': np.array(classification_list)
                }

        yield x_batch, y_batch

for x, y in keras_generator_train_val_test(2, choice="train"):
    break

print(x.shape, y['seg'].shape)